In [0]:
# Here we'll predict if a member of Congress is a Democrat or a Republican
# This is an example of Binary Classifiaction
# This is where someone is A OR B. Also good for CNN (Cat OR Dog)

In [0]:
import pandas as pd
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [4]:
from google.colab import files
uploaded = files.upload()
uploaded
!ls

Saving house-votes-84.data.txt to house-votes-84.data.txt
Saving house-votes-84.names.txt to house-votes-84.names.txt
house-votes-84.data.txt  house-votes-84.names.txt  sample_data


In [15]:
# Load the dataset (From 1984!)
columnNames = ['party','handicapped-infants', 'water-project-cost-sharing', 
                    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                    'el-salvador-aid', 'religious-groups-in-schools',
                    'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                    'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                    'education-spending', 'superfund-right-to-sue', 'crime',
                    'duty-free-exports', 'export-administration-act-south-africa']

votingData = pd.read_csv('house-votes-84.data.txt', na_values=['?'], names = labels)
votingData.head()


,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [16]:
# More in-depth statistics
votingData.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [17]:
# We can see that some politicians abastained so in this case we'll drop this NaN's
# Normally we run the risk of bias here, but for now it's OK
votingData.dropna(inplace=True)
votingData.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


In [18]:
# Normalise the data
votingData.replace(('y','n'),(1,0),inplace=True)
votingData.replace(('democrat','republican'),(1,0),inplace=True)
votingData.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


In [0]:
# Extract features and labels
features = votingData[columnNames].values
labels = votingData['party'].values

In [0]:
# Build our model
def predictParty():
  model = Sequential()
  # input_dim is 16 as there are 16 issues being voted on.
  # kernel_initializer sets the initial random weight
  model.add(Dense(32, input_dim=17,kernel_initializer='normal',activation='relu'))
  # Reduce 32 neurons to 16 to spread out learning
  model.add(Dense(16,kernel_initializer='normal',activation='relu'))
  # Output layer (Note the diff as it's binary)
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  # Compile the model
  model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
  return model

In [32]:
# Wrap model in estimator and check accuracy.
# Build function = create a model
estimate = KerasClassifier(build_fn=predictParty,epochs=100,verbose=0)
# Evaluate the accuracy
accuracyScore = cross_val_score(estimate,features,labels,cv=10)
accuracyScore.mean()

1.0